<a href="https://colab.research.google.com/github/Einav980/DataSciencesHIT/blob/main/DataCleaning_DataOrgenaize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import sklearn
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score

In [ ]:
#copy DF for cleaning 
cleanDf = pd.read_csv("DataFrame-withGlassDoor.csv").copy()

In [ ]:
#Identify empty values and replace with none
isNumeric = cleanDf.select_dtypes(include='number')
isValues = cleanDf.select_dtypes(exclude='number')

for col in isNumeric:
    cleanDf[col].replace({-1: None}, inplace=True)
for col in isValues:
    cleanDf[col].replace({'-1': None}, inplace=True)
    cleanDf[col].replace({'-': None}, inplace=True)
    cleanDf[col].replace({'Unknown / Non-Applicable': None}, inplace=True)
    cleanDf[col].replace({'Unknown': None}, inplace=True)


In [ ]:
#Delete rows with less data
cleanDf.drop(['rating_count', 'Unnamed: 0', 'company_name' , 'Headquarters'] , axis=1, inplace= True )
cleanDf.dropna(axis=0 ,subset=['rating', 'EPS_this_Y'], inplace= True ) 
cleanDf.dropna(axis=0 ,thresh = 10, inplace= True)

In [ ]:
#remove duplicate rows 
cleanDf.drop_duplicates(inplace=True)

In [ ]:
#fill empty cells
marketTop = cleanDf['Market_Cap'].describe(exclude= None).top
EPSNextTop = cleanDf['EPS_next_Y'].describe(exclude= None).top
GlobalSizeTop = cleanDf['Global_Company_Size'].describe(exclude= None).top
FoundedTop = cleanDf['Founded'].describe(exclude= None).top
RevenueTop = cleanDf['Revenue'].describe(exclude= None).top
RecFriendTop = cleanDf['Recommend_to_friend'].describe(exclude= None).top
CEOAppTop = cleanDf['Approve_of_CEO'].describe(exclude= None).top

cleanDf.fillna({'P_E' : -1 ,'Market_Cap': marketTop, 'EPS_next_Y': EPSNextTop , 'Global_Company_Size' : GlobalSizeTop, 'Founded' : FoundedTop, 'Revenue' :RevenueTop, 'Recommend_to_friend' : RecFriendTop, 'Approve_of_CEO' : CEOAppTop}, inplace=True)


In [ ]:
#Df after cleaning
cleanDf.to_csv("DataFrame_AfterCleaning.csv")

In [ ]:
#get DF for orgenaizing
orgenizeDf = pd.read_csv("DataFrame_AfterCleaning.csv").copy()

In [ ]:
#convert strings to numerial values
size_dic = {"M": 1000000,"B": 1000000000,"K": 1000, "%" : 0.01}
orgenizeDf["Recommend_to_friend"]= orgenizeDf["Recommend_to_friend"].astype(float)
orgenizeDf["Approve_of_CEO"]= orgenizeDf["Approve_of_CEO"].astype(float)

for i in range(orgenizeDf.shape[0]):
    orgenizeDf["Market_Cap"][i]= float(orgenizeDf["Market_Cap"][i][:-1]) * size_dic[orgenizeDf["Market_Cap"][i][-1]]
    orgenizeDf["Avg_Volume"][i]= float(orgenizeDf["Avg_Volume"][i][:-1]) * size_dic[orgenizeDf["Avg_Volume"][i][-1]]
    orgenizeDf["EPS_this_Y"][i]= float(orgenizeDf["EPS_this_Y"][i][:-1]) * size_dic[orgenizeDf["EPS_this_Y"][i][-1]]
    orgenizeDf["EPS_next_Y"][i]= float(orgenizeDf["EPS_next_Y"][i][:-1]) * size_dic[orgenizeDf["EPS_next_Y"][i][-1]]
    orgenizeDf["Recommend_to_friend"][i]= orgenizeDf["Recommend_to_friend"][i] *0.01
    orgenizeDf["Approve_of_CEO"][i] = orgenizeDf["Approve_of_CEO"][i] *0.01


<ipython-input-16-88e2f0f9562c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orgenizeDf["Market_Cap"][i]= float(orgenizeDf["Market_Cap"][i][:-1]) * size_dic[orgenizeDf["Market_Cap"][i][-1]]
<ipython-input-16-88e2f0f9562c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orgenizeDf["Avg_Volume"][i]= float(orgenizeDf["Avg_Volume"][i][:-1]) * size_dic[orgenizeDf["Avg_Volume"][i][-1]]
<ipython-input-16-88e2f0f9562c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [ ]:
#split share_name_index column to 2 colomns
snp500 = []
index = []
# shareName =[]

for i in range(orgenizeDf.shape[0]):
    tmp= (orgenizeDf["share_name_index"][i].split(sep= " [" , maxsplit = 1)[1][:-1].split(sep= ", " , maxsplit = 1))
    index.append(tmp[0])
#     shareName.append(orgenizeDf["share_name_index"][i].split(sep= " [" , maxsplit = 1)[0])
    if (len(tmp)== 2):
        snp500.append(1)
    else:
        snp500.append(0)

orgenizeDf["inSNP500"] = snp500
orgenizeDf["Index"] = index
# orgenizeDf['share_name']= shareName
orgenizeDf.drop(['share_name_index','Unnamed: 0'],axis='columns', inplace=True)

In [ ]:
#change to categorial
le = preprocessing.LabelEncoder()

#change str columns to categorial 
orgenizeDf['Index']=le.fit_transform(orgenizeDf['Index'])
orgenizeDf['Industry']=le.fit_transform(orgenizeDf['Industry'])
orgenizeDf['Global_Company_Size']=le.fit_transform(orgenizeDf['Global_Company_Size'])
orgenizeDf['Revenue']=le.fit_transform(orgenizeDf['Revenue'])


orgenizeDf['P_E'] = orgenizeDf['P_E'] > 0
orgenizeDf['P_E']=le.fit_transform(orgenizeDf['P_E'])

orgenizeDf['contry'] = orgenizeDf['contry'] == "USA"
orgenizeDf['contry']=le.fit_transform(orgenizeDf['contry'])

orgenizeDf['Approve_of_CEO'] = orgenizeDf['Approve_of_CEO']> 0.75
orgenizeDf['Recommend_to_friend'] = orgenizeDf['Recommend_to_friend']> 0.75
orgenizeDf['Approve_of_CEO']=le.fit_transform(orgenizeDf['Approve_of_CEO'])
orgenizeDf['Recommend_to_friend']=le.fit_transform(orgenizeDf['Recommend_to_friend'])


# #change numeric columns to categorials
# bins = [1000000,10000000,100000000,1000000000,10000000000,100000000000,1000000000000,10000000000000]
# labels = [1,2,3,4,5,6,7]
# orgenizeDf['Market_Cap'] = pd.cut(orgenizeDf['Market_Cap'], bins, labels=labels)

# bins = [100,1000,10000,100000,1000000,10000000,100000000,1000000000]
# labels = [1,2,3,4,5,6,7]
# orgenizeDf['Avg_Volume'] = pd.cut(orgenizeDf['Avg_Volume'], bins, labels=labels)

# bins = [1600,1700,1800,1900,1950,2000,2050]
# labels = [1,2,3,4,5,6]
# orgenizeDf['Founded'] = pd.cut(orgenizeDf['Founded'], bins, labels=labels)

# bins = [-1,0.1,0.2,0.3,0.4,0.5,0.6, 0.7, 0.8, 0.9,1]
# labels = [1,2,3,4,5,6,7,8,9,10]
# orgenizeDf['Approve_of_CEO'] = pd.cut(orgenizeDf['Approve_of_CEO'], bins, labels=labels)
# orgenizeDf['Recommend_to_friend'] = pd.cut(orgenizeDf['Recommend_to_friend'], bins, labels=labels)

# bins = [-10,-5,-1,0,1,5,10,18,26]
# labels = [1,2,3,4,5,6,7,8]
# orgenizeDf['EPS_this_Y'] = pd.cut(orgenizeDf['EPS_this_Y'], bins, labels=labels)

# bins = [-10,-5,-1,0,1,5,10,75,150,250]
# labels = [1,2,3,4,5,6,7,8,9]
# orgenizeDf['EPS_next_Y'] = pd.cut(orgenizeDf['EPS_next_Y'], bins, labels=labels)


In [ ]:
#dumies
orgenizeDf= pd.get_dummies(orgenizeDf, columns = ['Index'])

In [ ]:
orgenizeDf.to_csv("text.csv")

In [ ]:
orgenizeDf.shape

(3488, 17)